In [14]:
import pandas as pd
import io
import re
from os.path import join

In [15]:

def get_genome_data(genome_file):
    genome_data = pd.read_csv(
        genome_file,
        comment='#',
        sep='\t',
        header=None,
        names=['rsid', 'chromosome', 'position', 'genotype']
    )
    genome_data = genome_data.rename(columns={
        'chromosome': 'Chromosome',
        'position': 'Position',
        'genotype': 'Genotype'
    })
    return genome_data


def _risk_genotype_match(row):
    # Extract risk genotypes from the 'Risk Genotypes' column
    risk_genotypes = re.findall(
        r'\b\w+\b', str(row['Risk Genotypes']))
    user_genotype = row['Genotype']
    return user_genotype in risk_genotypes


def get_risk_genotypes(genome_data):
    # Read in snp_mapping
    snp_mapping = pd.read_csv(join('data', 'mapping.csv'))
    snp_mapping = snp_mapping.drop(columns=['Verified', 'Source'])
    # Merge user genome_data with snp_mapping
    merged_data = pd.merge(
        genome_data, snp_mapping, on='rsid', how='inner')
    # Match risk genotypes
    filtered_data = merged_data[merged_data.apply(
        _risk_genotype_match, axis=1)]
    return filtered_data

In [16]:
genome_data = get_genome_data('23andMe.txt')
risk_genome_data = get_risk_genotypes(genome_data)
risk_genome_data

C:\Users\greys\AppData\Local\Temp\ipykernel_32392\4175101056.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  genome_data = pd.read_csv(


,rsid,Chromosome,Position,Genotype,Gene,Normal Genotypes,Risk Genotypes,Risk Description,Function,Affected Ingredients,Alternative Ingredients
7,rs1126809,11,88657609,GG,TYR,AA,"GG,AG,GA","Altered melanin production, potentially leadin...",Melanin production,Hydroquinone (skin-lightening agents),"Tranexamic acid, licorice root extract, alpha ..."
13,rs3810280,19,55912749,CC,CRB1,TT,"CT,TC,CC","Increased susceptibility to oxidative stress, ...",Cellular structure and integrity,"Harsh exfoliants, pro-oxidant treatments","Antioxidants (vitamin C, E), gentle skincare r..."
